# 21. 연산자 오버로드(2)

- 개선점: ndarray 인스터스 및 수치 데이터와 함께 사용하기
- Variable 인스턴스, ndarry 인스턴스, int, float 등도 함께 사용할 수 있게 수정해 봅시다. 

## 21.1 ndarray와 함께 사용하기

- a * np.array(2.0) 코드를 만나면 ndarray 인스턴스를 자동으로 Variable 인스턴스로 변환하기
- as_variable 편의 함수를 만들어서 인수로 주어진 객체를 Variable 인스턴스로 변환해 주는 함수 구현하기

In [1]:
def as_variable(obj):
    if isinstance(obj, Variable):    # 인수 obj가 Variable 인스턴스면 그대로 반환
        return obj
    return Variable(obj)   # 그렇지 않으면 Variable 인스턴스로 변환하여 반환

In [2]:
class Function: # DeZero에서 사용하는 모든 함수(연산)은 Function 클래스를 상속하므로 실제 연산은 Function __call__메서드에서 이뤄짐
    def __call__(self, *inputs):   # 따라서 __call__메서드에 가한 수정은 DeZero에서 사용하는 모든 함수에 적용됨
        inputs = [as_variable(x) for x in inputs]   # 인수 inputs에 담긴 각각의 원소 x를 Variable 인스턴스로 변환

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)

In [5]:
x = Variable(np.array(2.0))    # ndarray 인스턴스가 Variable 인스턴스로 자동 변환 됨 
y = x + np.array(3.0)          # 이제 ndarray와 Variable 함께 사용할 수 있게 됨
print(y)

variable(5.0)


## 21.2 float, int와 함께 사용하기

- 파이썬의 float와 int, 그리고 np.float64과 np.int64 같은 타입과도 함께 사용할 수 있도록 하겠습니다. 
- x가 Variable 인스턴스일 때, x + 3.0 같은 코드를 실행할 수 있도록 하려면 어떻게 해야 할까요?

In [6]:
def add(x0, x1):
    x1 = as_array(x1)      # x1이 float나 int일 경우, ndarray 인스턴스로 변환 
    return Add()(x0, x1)   # 이후 ndarray 인스턴스는 Function 클래스에서 Variable 인스턴스로 변환

In [7]:
x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


- 이와 같이 float와 Variable 인스턴스를 조합한 계산이 가능해짐
- add 함수 외에도 mul 같은 다른 함수도 같은 방식으로 수정할 수 있음
- 수정하고 나면 +나 *로 Variable 인스턴스, float, int를 조합하여 계산할 수 있음
- 지금의 방식에는 두 가지 문제가 남아 있음

## 21.3 문제점 1: 첫번째 인수가 float나 int인 경우

- 현재 DeZero는 x * 2.0을 제대로 실행할 수 있지만 2.0 * x 실행하면 오류가 납니다. 
    - 연산자 왼쪽에 있는 2.0의 \_\_mul\_\_ 메서드를 호출하려 시도한다
    - 하지만 2.0은 float 타입이므로 \_\_mul\_\_ 메서드는 구현되어 있지 않다. 
    - 다음은 * 연산자 오른쪽에 있는 x의 특수 메서드를 호출하려 시도한다
    - x가 오른쪽에 있기 때문에 (\_\_mul\_\_ 메서드 대신) \_\_rmul\_\_ 메서드를 호출하려 시도한다.
    - 하지만 Variable 인스턴스에는 \_\_rmul\_\_ 메서드가 구현되어 있지 않다. 
- 핵심은 * 같은 이항 연산자의 경우 피연산자(항)의 위치에 따라 호출되는 특수 메서드가 다르다는 것입니다. 
    - 곱셉의 경우 피연산자가 좌항이면 \_\_mul\_\_ 메서드가 호출되고, 우항이면 \_\_rmul\_\_ 메서드가 호출됩니다. 
- 따라서 이번 문제는 \_\_rmul\_\_ 메서드를 구현하면  해결됩니다. 
    - 이 때 \_\_rmul\_\_ 메서드의 인수는 아래처럼 전달됩니다. 
    
![title](./image/그림21-1.png)

- 위 그림과 같이 \_\_rmul\_\_(self, other)의 인수 중 self는 자신인 x에 대응하고, other는 다른 쪽 항이 2.0에 대응합니다. 
- 곱셈에서는 좌항과 우항을 바꿔도 결과가 같기 때문에 둘을 구별할 필요가 없습니다. 
- 덧셉도 마찬가지이므로 +와 $*$의 특수 메서드는 아래처럼 설정하면 됩니다. 

In [9]:
Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

In [10]:
x = Variable(np.array(2.0)) # 이제 Variable 인스턴스와 float, int를 함께 사용할 수 있습니다. 
y = 3.0 * x + 1.0
print(y)

variable(7.0)


## 21.4 문제점 2: 좌항이 ndarray 인스턴스인 경우

- 남은 문제는 ndarray 인스턴스가 좌항이고 Variable 인스턴스가 우항인 경우입니다. 

In [11]:
x = Variable(np.array(1.0)) # 좌항 ndarray 인스턴스의 __add__ 메서드가 호출되는데, 
y = np.array([2.0]) + x     # 우항인 Variable 인스턴스의 __radd__ 메서스가 호출되길 원함. 

In [12]:
# 그러려면 '연산 우선순위'를 지정해야 함. Variable 인스턴스 속성에 __array_priority__를 추가하고 그 값을 큰 정수로 설정함
class Variable:               # Variable 인스턴스의 연산자 우선순위를 ndarray 인스턴스 연산자 우선순위보다 높일 수 있다. 
    __array_priority__ = 200  # 그 결과 좌항이 ndarray 인스턴스라 해도 우항인 Variable 인스턴스의 연산자 메서드가 우선적으로 호출됨

In [13]:
# 21장 전체 코드

import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul

x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

y = x + 3.0
print(y)

y = 3.0 * x + 1.0
print(y)

variable(5.0)
variable(5.0)
variable(7.0)


# 22. 연산자 오버로드(3)

- 이번 단계에서는 아래 연산자들을 새로 추가하겠습니다. 

![title](image/표22-1.png)

- 단항 연산자
- 이항 연산자의 경우 우항/좌항 구분
- 거듭제곱의 경우 좌항이 Variable 변수임
- 새로운 연산자를 추가하는 순서
    - Function 클래스를 상속하여 원하는 함수 클래스를 구현합니다. (예: Mul 클래스)
    - 파이썬 함수를 사용할 수 있도록 합니다. (예: mul 함수)
    - Variable 클래스의 연산자를 오버로드 합니다. (예: Variable, \_\_mul\_\_=mul)
- 이번 단계에서도 똑같은 과정으로 새로운 연산자를 추가합니다. 

## 22.1 음수(부호 변환)

- 음수의 미분은 y = -x 일때, $dy/dx = -1$ 입니다. 
- 따라서 역전파는 상류(출력 쪽)에서 전해지는 미분에 -1을 곱하여 하류로 흘려보내 주면됩니다. 

In [14]:
class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)

Variable.__neg__ = neg

- 위와 같이 Neg 클래스를 구현한 다음, 파이썬 함수로 사용할 수 있도록 neg 함수도 구현합니다. 
- 그리고 특수 메서드인 \_\_neg\_\_에 neg를 대입하면 완성입니다. 

In [15]:
x = Variable(np.array(2.0))
y = -x
print(y)  # variable(-2.0)

variable(-2.0)


## 22.2 뺄셈

- 뺄셈의 미분은 $y = x_0 - x_1$ 일 때 $dy/dx_0=1, dy/dx_1=1$ 입니다. 
- 따라서 역전파는 상류에서 전해지는 미분값에 1을 곱한 값이 $x_0$ 의 미분값이 되며, -1을 곱한 값이 $x_1$의 미분결과가 됩니다. 

In [16]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

Variable.__sub__ = sub

- y = 2.0 - x 같은 코드는 제대로 처리되지 않습니다. 
- x의 \_\_rsub\_\_메소드가 호출됩니다. 
- \_\_rsub\_\_(self, other)가 호출될 때는 우항인 x가 인수 self에 전달됩니다.

In [ ]:
def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)    # x0와 x1의 순서를 바꾼다.

Variable.__rsub__ = rsub

In [19]:
y1 = 2.0 - x
y2 = x - 1.0
print(y1)  # variable(0.0)
print(y2)  # variable(1.0)

variable(0.0)
variable(1.0)


## 22.3 나눗셈

- 나눗셈의 미분은 $y = x_0/x_1$ 일 때 $dy/dx_0=1/x_1, dy/dx_1=-x_0/(x_1)^2$ 입니다. 
- 나눗셈도 뺄셈과 마찬가지로 좌/우항 중 어느 것에 적용할지에 다라 적용되는 함수가 다릅니다. 

In [20]:
class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)   # x0와 x1의 순서를 바꾼다.

Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv

## 22.4 거듭제곱

- 거듭제곱은 $y = x^c$ 형태로 표현됩니다. 밑이 x이고 지수가 c입니다. 
- 밑이 x인 경우만 미분하면 $dy/dx=cx^{c-1}$이 됩니다. 
- 지수 c는 상수로 취급하여 따로 미분 계산하지 않습니다. 

In [21]:
class Pow(Function):
    def __init__(self, c):   # 클래스를 초기화 할 때 지수 c를 제공할 수 있습니다. 
        self.c = c

    def forward(self, x):    # 순전파 메소드에서는 밑에 해당하는 x만 (즉, 하나의 항만) 받게 됩니다. 
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)

Variable.__pow__ = pow    # 특수 메소드인 __pow__에 함수 pow를 할당합니다. 

- 이제 $**$ 연산자를 사용하여 거듭 제곱을 계산할 수 있습니다. 

In [24]:
x = Variable(np.array(2.0))
y = x ** 3
print(y)  # variable(8.0)

variable(8.0)


- 이상으로 목표한 연산자를 모두 추가하여 사칙연산자들을 자유롭게 계산에 활용할 수 있게 되었습니다. 
- 다음 단계에서 지금까지 성과를 파이썬 패키지로 정리한 다음 DeZero의 실력을 검증해 보겠습니다. 

In [27]:
import weakref
import numpy as np
import contextlib


class Config:
    enable_backprop = True


@contextlib.contextmanager
def using_config(name, value):
    old_value = getattr(Config, name)
    setattr(Config, name, value)
    try:
        yield
    finally:
        setattr(Config, name, old_value)


def no_grad():
    return using_config('enable_backprop', False)


class Variable:
    __array_priority__ = 200

    def __init__(self, data, name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name
        self.grad = None
        self.creator = None
        self.generation = 0

    @property
    def shape(self):
        return self.data.shape

    @property
    def ndim(self):
        return self.data.ndim

    @property
    def size(self):
        return self.data.size

    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)

    def __repr__(self):
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self, retain_grad=False):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        if Config.enable_backprop:
            self.generation = max([x.generation for x in inputs])
            for output in outputs:
                output.set_creator(self)
            self.inputs = inputs
            self.outputs = [weakref.ref(output) for output in outputs]

        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0


def mul(x0, x1):
    x1 = as_array(x1)
    return Mul()(x0, x1)


class Neg(Function):
    def forward(self, x):
        return -x

    def backward(self, gy):
        return -gy


def neg(x):
    return Neg()(x)


class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y

    def backward(self, gy):
        return gy, -gy


def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)


def rsub(x0, x1):
    x1 = as_array(x1)
    return sub(x1, x0)


class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1


def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)


def rdiv(x0, x1):
    x1 = as_array(x1)
    return div(x1, x0)


class Pow(Function):
    def __init__(self, c):
        self.c = c

    def forward(self, x):
        y = x ** self.c
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c

        gx = c * x ** (c - 1) * gy
        return gx


def pow(x, c):
    return Pow(c)(x)


Variable.__add__ = add
Variable.__radd__ = add
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__neg__ = neg
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__truediv__ = div
Variable.__rtruediv__ = rdiv
Variable.__pow__ = pow

x = Variable(np.array(2.0))
y = -x
print(y)  # variable(-2.0)

y1 = 2.0 - x
y2 = x - 1.0
print(y1)  # variable(0.0)
print(y2)  # variable(1.0)

y = 3.0 / x
print(y)  # variable(1.5)

y = x ** 3
y.backward()
print(y)  # variable(8.0)

variable(-2.0)
variable(0.0)
variable(1.0)
variable(1.5)
variable(8.0)


# 23. 패키지로 정리

- 이번 단계에서는 지금까지의 성과를 재사용할 수 있도록 패키지로 정리할 생각입니다. 
- 파이썬에서는 '모듈', '패키지', '라이브러리'라는 용어를 아래 의미로 사용합니다. 
    - 모듈: 파이썬 파일, 특히 다른 파이썬 프로그램에서 임포트하여 사용하는 것을 가정하고 만들어진 파이썬 파일을 모듈이라고 합니다. 
    - 패키지: 여러 모듈을 묶은 것. 패키지를 만들려면 먼저 디렉토리를 만들고 그 안에 모듈(파이썬 파일)을 추가합니다. 
    - 라이브러리: 여러 패키지를 묶은 것. 하나 이상의 디렉토리로 구성됨. 때로는 패키지를 가리켜 라이브러리라고 부르기도 합니다. 

## 23.1 파일 구성

- 지금까지는 각 step 파일에 코드를 작성했습니다. 
- 이제부터는 이 step 파일 모두에서 DeZero를 이용할 수 있도록 dezero라는 공통의 디렉토리 하나 만들겠습니다. 
- 최종 파일 구성은 다음과 같습니다. 
    - dezero
        - \_\_init\_\_.py
        - core_simple.py
        - ...
        - util.py
    - step
        - step01.py
        - step02.py
        - ...
        - step60.py
- 이와 같이 구성한 뒤 dezero 디렉토리에 모듈을 추가하는 것입니다. 
- 그리하여 dezero라는 패키지가 만들어지는데, 이 패키지가 바로 우리가 만드는 프레임워크입니다. 
- 앞으로는 주로 dezero 디텔토리에 있는 파일들에 코드를 추가할 것입니다. 

## 23.2 코어 클래스로 옮기기

- dezero 디렉토리에 파일을 추가해보죠.
- 목표는 이전 단계의 step22.py 코드를 dezero/core_simple.py라는 코어(core) 파일로 옮기는 것입니다. 
    - step22.py에 정의된 클래스들을 코어 파일에 복사
    - step22.py에 정의된 파이썬 함수들도 코어 파일에 복사
    - Exp 클래스와 Sqaure 클래스, exp 함수와 sqaure 함수는 코어 파일에 넣지 않고 나중에 dezero/function.py에 추가합니다. 

In [31]:
import numpy as np
from dezero.core_simple import Variable  # 외부의 파이썬 파일에서 다음과 같이 dezero를 임포트 할 수 있습니다. 

x = Variable(np.array(1.0))
print(x)

variable(1.0)


- from ... import ... 구문을 사용하면 모듈 내의 클래스나 함수 등을 직접 임포트 할 수 있습니다. 
- 또한 import XXX as A 라고 쓰면 XXX 라는 모듈을 A 라는 이름으로 임포트 할 수 있습니다. 
- 예를 들어 import dezero.core_simple as dz라고 스면 dezero.core_simple 모듈을 dz라는 이름으로 임포트 합니다. 
- 그런 다음 Variable 클래스를 사용하려면 dz Variable이라고 쓰면 됩니다. 

## 23.3 연산자 오버로드

- 오버로드한 연산자들을 dezero로 옮기겠습니다. 
- 코어 파일에 다음 함수들을 추가합니다. 

In [32]:
def setup_variable():        # Variable 연산자들을 오버로드 해주는 함수입니다. 이 함수를 호출하면 Variable의 연산자들이 설정됩니다. 
    Variable.__add__ = add
    Variable.__radd__ = add
    Variable.__mul__ = mul
    Variable.__rmul__ = mul
    Variable.__neg__ = neg
    Variable.__sub__ = sub
    Variable.__rsub__ = rsub
    Variable.__truediv__ = div
    Variable.__rtruediv__ = rdiv
    Variable.__pow__ = pow

- 이 함수는 어디에서 호출하면 좋을까요? 바로 dezero/\_\_init\_\_.py 파일입니다. 
- \_\_init\_\_.py는 모듈을 임포트할 때 가장 먼저 실행되는 파일입니다. 
- dezero 패키지에 속한 모듈을 임포트할 때 dezero/\_\_init\_\_.py의 코드가 첫 번째로 호출됩니다. 
- 그래서 dezero/\_\_init\_\_.py에 다음 코드를 작성해 넣어야 합니다. 

In [ ]:
from dezero.core_simple import Variable
from dezero.core_simple import Function
from dezero.core_simple import using_config
from dezero.core_simple import no_grad
from dezero.core_simple import as_array
from dezero.core_simple import as_variable
from dezero.core_simple import setup_variable

setup_variable()

- 이와 같이 setup_variable 함수를 임포트해 호출하도록 합니다. 
- 이렇게 함으로써 dezero 패키지를 이용하는 사용자는 반드시 연산자 오버로드가 이루어진 상태에서 Variable을 사용할 수 있습니다. 
- 한편 \_\_init\_\_.py 시작이 from dezero.core_simple import Variable 인데, 이 문장이 실행됨으로써 dezero 패키지에서 Variable 클래스를 곧바로 임포트 할 수 있습니다. 
- 지금까지 from dezero.core_simple import Variable 작성한 것을 from dezero import Variable 처럼 짧게 줄일 수 있습니다. 
- 마찬가지로 dezero/\_\_init\_\_.py의 임포트 덕분에 사용자는 나머지 Function이나 using_config 등도 '간소화된' 임포트를 이용할 수 있게 됩니다. 

In [ ]:
# dezero를 이용하는 사용자의 코드

# from dezero.core_simple import Variable
from dezero import Variable                  # dezero/__init__.py의 임포트 덕분에 위를 아래처럼 짧게 줄일 수 있습니다. 

## 23.4 실제 __init__.py 파일

- 실제 dezero/\_\_init\_\_.py 에서는 core_simple.py와 core.py 중 하나를 선택해 임포트하도록 작성되어 있습니다. 

In [ ]:
# =============================================================================
# step23.py부터 step32.py까지는 simple_core를 이용해야 합니다.
is_simple_core = False  # True
# =============================================================================

if is_simple_core:
    from dezero.core_simple import Variable
    from dezero.core_simple import Function
    from dezero.core_simple import using_config
    from dezero.core_simple import no_grad
    from dezero.core_simple import as_array
    from dezero.core_simple import as_variable
    from dezero.core_simple import setup_variable

else:
    from dezero.core import Variable
    from dezero.core import Parameter
    from dezero.core import Function
    from dezero.core import using_config
    from dezero.core import no_grad
    from dezero.core import test_mode
    from dezero.core import as_array
    from dezero.core import as_variable
    from dezero.core import setup_variable
    from dezero.core import Config
    from dezero.layers import Layer
    from dezero.models import Model
    from dezero.datasets import Dataset
    from dezero.dataloaders import DataLoader
    from dezero.dataloaders import SeqDataLoader

    import dezero.datasets
    import dezero.dataloaders
    import dezero.optimizers
    import dezero.functions
    import dezero.functions_conv
    import dezero.layers
    import dezero.utils
    import dezero.cuda
    import dezero.transforms

setup_variable()
__version__ = '0.0.13'

## 23.5 dezero 임포트하기

- 이렇게 하여 dezero라는 패키지가 만들어졌습니다. 
- 이제 이번 단계용 step23.py는 다음처럼 작성할 수 있습니다. 

In [45]:
# Add import path for the dezero directory.
if '__file__' in globals():   # 터미널에서 python 명령으로 실행하면 __file__변수가 정의되어 있음. 
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..')) # 현재 파일의 부모 디렉토리를 모듈 검색 경로에 추가함

import numpy as np
# from dezero.core_simple import Variable
from dezero import Variable


x = Variable(np.array(1.0))
y = (x + 3) ** 2
y.backward()

print(y)
print(x.grad)

variable(16.0)
8.0


# 24. 복잡한 함수의 미분

- DeZero는 대표적인 연산자들을 지원합니다. 평소 파이썬 프로그래밍 하듯 코딩할 수 있습니다. 
- 이 혜택은 복잡한 수식을 코딩할 때 피부로 느껴질 것입니다. 
- 이번 단계에서는 지금까지 성과를 느낄 수 있는 복잡한 수식의 미분 몇 가지를 풀어보겠습니다. 
- 이번 단계에서 다루는 함수들을 최적화 문제에서 자주 사용되는 테스트 함수입니다. 
- 최적화 문제의 테스트 함수란 다양한 최적화 기법이 '얼마나 좋은가'를 평가하는 데 사용되는 함수를 뜻합ㄴ디ㅏ. 
- 소위 '벤치마크'용 함수라고 할 수 있습니다. 
- 테스트 함수에도 종류가 많은데, 위키백과의 'Test functions for optimization' 페이지를 보면 대표적인 예를 확인할 수 있습니다. 

![title](image/그림24-1.png)


- 위는 일부만 발췌한 것이며, 우리는 이 중 세 함수를 선택하여 실제로 미분해보려 합니다. 
- 그러면 DeZero의 실력이 어느 정도인지 알 수 있겠죠.
- 우선 Sphere라는 간단한 함수에서 시작하겠습니다. 

## 24.1 Sphere 함수

- Sphere 함수를 수식으로 표현하면 $z = x^2 + y^2$ 입니다. 
- 우리가 할 일은 그 미분을 계산하는 것입니다. 
- 아래를 계산해 보면 미분 결과가 잘 나오는 것을 볼 수 있습니다. 

In [53]:
import numpy as np
# from dezero.core_simple import Variable
from dezero import Variable

def sphere(x, y):     # 차원이 x, y, z 뿐인 3차원 공간에서 Sphere 함수입니다. 일반적인 Sphere 함수의 수식은 위의 표에 나와있습니다. 
    z = x ** 2 + y ** 2    
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = sphere(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

2.0 2.0


## 24.2 matyas 함수

- 이어서 matyas('마차시' 라고 읽음) 함수를 살펴보겠습니다. 
- $ z = 0.26(x^2 + y^2) - 0.48xy $ 이며 아래처럼 구현할 수 있습니다. 

In [54]:
def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z

x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = matyas(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

0.040000000000000036 0.040000000000000036


- 수식을 그대로 코드로 옮길 수 있습니다. 
- 만약 이 연산자들을 사용할 수 없다면 아래와 같이 작성해야 합니다. 

In [57]:
def matyas(x, y):
    z = sub(mul(0.26, add(pow(x, 2), pow(y, 2))), mul(0.48, mul(x, y)))
    return z

## 24.3 Goldstein_Price 함수

- Goldstein_Price 함수를 수식으로 표현하면 다음과 같습니다. (p 198)

$ f(x,y) = [1 + (x + y + 1)^2(19 - 14x + 3x^2 - 14y + 6xy + 3y^2)] $
$          [30 + (2x - 3y)^2(18 - 32x + 12x^2 + 48y - 36xy + 27y^2)] $

In [58]:
def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z

- 수식과 비교해가며 코드로 옮기면 금방 끝날 것입니다. 
- 반면 이 연산자들을 사용하지 않고 코딩하기란 보통 사람들에게는 불가능할지 모릅니다. 
- 그럼, 이 함수를 미분해 볼까요?

In [59]:
x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

-5376.0 8064.0


- DeZero는 위와 같이 복잡한 계산도 훌륭하게 미분할 수 있습니다. 
- 또한 이 결과가 맞는지는 기울기 확인으로 검증할 수 있습니다. 
- 이것으로 두번째 고지도 무사히 정복했습니다. 

## 제 2고지 정리

- 이제 DeZero는 복잡한 계산도 가능하게 되었습니다. 
- 엄밀히 말하면 아무리 복잡하게 '연결'된 계산 그래프라도 올바르게 역전파 할 수 있습니다. 
- 또한 연산자를 오버로드한 덕에 보통의 파이썬 프로그래밍처럼 코드를 작성할 수 있습니다. 
- 일반적인 파이썬 연산자를 이용해도 미분을 자동으로 계산할 수 있기 때문에 DeZero는 '일반적인 프로그래밍'을 '미분 가능'하게 만들었다고 표현할 수도 있습니다. 
- 다음 단계부터는 더 고급 계산도 처리할 수 있도록 DeZero를 확장해갈 것입니다. 

In [ ]:
if '__file__' in globals():
    import os, sys
    sys.path.append(os.path.join(os.path.dirname(__file__), '..'))
import numpy as np
from dezero import Variable


def sphere(x, y):
    z = x ** 2 + y ** 2
    return z


def matyas(x, y):
    z = 0.26 * (x ** 2 + y ** 2) - 0.48 * x * y
    return z


def goldstein(x, y):
    z = (1 + (x + y + 1)**2 * (19 - 14*x + 3*x**2 - 14*y + 6*x*y + 3*y**2)) * \
        (30 + (2*x - 3*y)**2 * (18 - 32*x + 12*x**2 + 48*y - 36*x*y + 27*y**2))
    return z


x = Variable(np.array(1.0))
y = Variable(np.array(1.0))
z = goldstein(x, y)  # sphere(x, y) / matyas(x, y)
z.backward()
print(x.grad, y.grad)

## 칼럼: Define_by_Run

- 딥러닝 프레임워크는 동작 방식에 따라 2가지로 나눌 수 있습니다. 

### Define-and-Run(정적 계산 그래프 방식)

- (사용자가) 계산그래프를 정의한 다음 (프레임워크는 주어진 그래프를 컴퓨터가 처리할 수 있는 형태로 변환하여) 데이터를 흘려보내는 방식

![title](image/그림B-1.png)

- 컴파일: 프레임워크는 계산 그래프의 정의를 변환하는 과정. 
- 컴파일 과정을 통해서 계산 그래프가 메모리상에 펼쳐지며, 실제 데이터를 흘려보낼 준비가 갖춰진다. 
- 딥러닝 프레임워크를 '미분 가능 프로그래밍(differentiable programming)'이라고도 함
- 텐서플로, 카페, CNTK가 있음 (텐서플로 2.0부터는 Define-by-Run 방식을 도입함. DeZero도 Define_by_Run 방식임)

#### 정적 계산 그래프의 단점

- '도메인 특화 언어'(파이썬 위에서 동작하는 새로운 프로그래밍 언어)를 사용하여 계산을 정의함
    - 프로그래머가 도메일 특화 언어를 새롭게 배워야 하는 부담이 생김
- 디버깅이 어려움
    - 프레임워크에서 컴파일을 거쳐 프레임워크만 이해하고 실행할 수 있는 표현 형식으로 변환되므로 파이썬 프로세서는 이 독자적인 표현방식을 이해할 수 없음
    - 버그는 '데이터 흘려보낼 때' 발견되지만, 원인은 '계산 그래프 정의'에 있는 경우가 대부분이라 디버깅이 어려움
    
####  정적 계산 그래프의 장점

- 가장 큰 장점은 성능임. 계산 그래프를 최적화 하면 성능도 따라서 최적화 됨
- 계산 그래프 최적화는 계산 그래프 구조와 사용되는 연산을 효율적인 것으로 변환하는 형태로 이루어짐
- 아래 최적화 버전에서는 곱셈과 덧셈을 한 번에 수행하는 연산을 사용함 (이 변환으로 두 개의 연산을 하나의 연산으로 축약하여 계산 시간이 단축됨)

![title](image/그림B-2.png)


- 데이터를 흘려보내기 전에 전체 계산 그래프가 손에 들어오므로 계산 그래프 전체를 고려해 최적화할 수 있음
    - 신경망 학습은 '신경망을 한 번만 정의하고 정의된 신경망에 데이터를 여러 번 흘려보내는' 형태로 활용됨
    - 따라서 신경망 구축과 최적화에 시간을 좀 더 들이면, 데이터를 반복해 흘려보내는 단계에서 시간을 절약할 수 있음
- 어떻게 컴파일하느냐에 따라 다른 실행 파일로 변환될 수 있음
    - 파이썬이 아닌 다른 환경에서도 데이터를 흘려보내는 것이 가능함(파이썬 자체가 주는 오버헤드가 사라지므로, IoT 기기처럼 자원이 부족한 에지 edge 전용 환경에서는 특히 중요한 특징임)
- 학습을 여러 대의 컴퓨터에 분산해 수행하는 경우에도 유리함
    - 계산 그래프 자체를 분할하여 여러 컴퓨터로 분배하는 시나리오는 사전에 계산 그래프가 구축되어 있어야만 가능하므로 정적 계산 그래프가 더 유리함

### Define-by-Run(동적 계산 그래프 방식)

- 데이터를 흘려보냄으로써 계산 그래프가 정의되는 방식
- 데이터를 흘려보내기와 계산 그래프 구축이 동시에 이루어지는 것이 특징임
    - DeZero의 경우 사용자가 데이터를 흘려보낼 때(일반적인 수치 계산을 수행할 때) 자동으로 계산 그래프를 구성하는 '연결(참조)'를 만듦
    - 구현 수준에서는 연결리스트로 표현되는데, 그 이유는 계산이 끝난 후 해당 연결을 역박향으로 추적할 수 있기 때문임
- 2015년 체이너에 의해 처음 만들어졌고, 파이토치, MXNet, DyNet, 텐서플로 (2.0 이상)에서 사용되는 방식

#### 동적 계산 그래프의 장점

- 프레임 워크 고유의 '도메일 특화 언어'를 배우지 않아도 됨
- 계산 그래프를 '컴파일'하여 독자적인 데이터 구조로 변환할 필요도 없음
- 일반적인 파이썬 프로그래밍으로 계산 그래프를 구축하고 실행할 수 있음 (파이썬의 if문이나 for문을 그대로 사용하여 계산 그래프를 만들 수 있음)
- 디버깅도 항상 파이썬 프로그램으로 할 수 있으므로, 디버깅도 유리함 (pdb 같은 파이썬 디버거를 사용할 수 있음)


### 두 방식의 비교

![title](image/표B-1.png)

- 성능이 중요할 때는 정적 계산 그래프 방식이, 사용성이 중요할 때는 동적 계산 그래프 방식이 유리함
- 두 모드를 모두 지원하는 프레임워크도 많음
    - 파이토치는 기본적으로 동적 계산 그래프 모드로 수행되지만, 정적 계산 그래프 모드도 제공함
    - 체이너도 기본은 동적 계산 그래프 모드이지만, 정적 계산 그래프 모드로 전환할 수 있음
    - 텐서플로 2.0도 역시 Eager Execution이라는 동적 계산 그래프가 표준으로 채택되지만, 필요 시 정적 계산 그래프로 전환 가능함
- 최근에는 프로그래밍 언어 자체에서 자동 미분을 지원하려는 시도를 볼 수 있음
    - Swift for TensorFlow가 대표적임. 스위프트 Swift라는 범용 프로그래밍 언어를 확장하여 (스위프트 컴파일러를 손질하여) 자동 미분 구조를 도입하려는 시도임
    - 자동 미분을 프로그래밍 언어 차원에서 지원하므로 성능과 사용성이라는 두 마리 토끼를 모두 잡을 수 있을 것이라 기대됨   